# Notebook notes

This notebook is for testing training

# Import libs

**setup autoreload**

In [1]:
%load_ext autoreload
%autoreload 2

**configure system path**

In [2]:
import os
import sys

# list directories where packages are stored
# note that the parent directory of te repo is added automatically
GYM_FOLDER = "gym-t99"

# get this notebook's current working directory
nb_cwd = os.getcwd()
# get name of its parent directory
nb_parent = os.path.dirname(nb_cwd)
# add packages to path
sys.path.insert(len(sys.path), nb_parent)
sys.path.insert(len(sys.path), os.path.join(nb_parent, GYM_FOLDER))

**import custom packages**

Use this code if you have not yet loaded gym_t99

In [3]:
import gym
registered = gym.envs.registration.registry.env_specs.copy()

import gym_t99
import t_net

**import custom packages**

Use this code if you have already loaded gym_t99, but now you introduced changes and want to re-load it again

In [11]:
import imp

# this code removes environment from gym's registry
env_dict = gym.envs.registration.registry.env_specs.copy()
for env in env_dict:
    if env not in registered:
        print("Remove {} from registry".format(env))
        del gym.envs.registration.registry.env_specs[env]

imp.reload(gym_t99)

Remove basic-v0 from registry
Remove t99-v0 from registry


<module 'gym_t99' from 'C:\\Users\\slavc\\Desktop\\UCLA\\Courses\\Spring_2021\\239AS\\239AS_T99_RL_Agent\\gym-t99\\gym_t99\\__init__.py'>

**import out-of-the-box packages**

In [4]:
# import matplotlib
import matplotlib as plt
# configure matplotlib
%matplotlib inline

In [5]:
import numpy as np
from collections import OrderedDict

In [6]:
# import torch
import torch
from torch import nn

# configure torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Workspace

In [73]:
# a simple random agent to play aginst and test
class RandomEnemy:
    def __init__(self):
        self.action_space = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    
    # this interface needs to be supported for any agent
    def action(self, state):
        return np.random.choice(self.action_space)

In [177]:
custom_gym = gym.make('gym_t99:t99-v0', num_players = 2, enemy=RandomEnemy())

In [181]:
frame = custom_gym.render(mode="debug")
print(frame[0])

[[10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  7  7  0  0 10 10]
 [10 10  0  0  0  0  0  0  7  7  0  0 10 10]
 [10 10  0

In [182]:
for _ in range(20):
    custom_gym.step(1)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


ValueError: operands could not be broadcast together with shapes (4,5) (5,5) 

In [183]:
# custom_gym.state.players[0].board[21, 5:12] = 1
print(custom_gym.state.players[1].board)
print(custom_gym.state.players[1].piece_current.matrix)

[[10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0

In [159]:
custom_gym.step(1)

(None, None, None, {})

In [151]:
a = [i for i in range(12, 1, -1)]
print(a)

[12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2]


In [ ]:
frame = custom_gym.render(mode="debug")
print(frame[0])